# Final Project, Part 2

The purpose of this assignment is to create a 'Viz for Experts' with an interactive dashboard interface for exploring your data.

For this submission option, you will submit your work through this Workspace.
    
**Please see Homework Prompt in PrairieLearn interface for more details on the requirements for this assignment.**

A rough outline of elements of code and write-up is shown below:

## Code:

 * An interactive dashboard within your Workspace that helps an expert explore your dataset thoroughly.
 * There should be a "dashboard" type aspect to this - i.e. a linked view exploring your dataset in an interactive way (like in Lab \#4) with [bqplot](https://bqplot.github.io/bqplot/).
 * Do not delete any cells, *just comment them out*. Show your work.



In [ ]:
# pip install folium

## Importing all the libraries

In [2]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
import folium
from IPython.display import display
import ipywidgets as widgets
from bqplot import Pie, Figure, Tooltip
from ipywidgets import Label, VBox, HBox

/home/jovyan/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ohkh3gml because the default path (/tmp/cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


## Cleaning the dataset

In [3]:

def download_dataset(api_url):
    response = requests.get(api_url)
    response.raise_for_status()  
    return response.text


def process_dataset(data_string):
    data = pd.read_csv(StringIO(data_string))
    
    
    
    replacements = {
        '<0.05': '0.05',
        '< 0.05': '0.05',
        '>0.95': '0.95',
        '> 0.95': '0.95',
        '-999': '0'
    }
    data.replace(replacements, regex=True, inplace=True)

    data['minor'] = data['minor'].astype(float)
    data['moderate'] = data['moderate'].astype(float)
    data['major'] = data['major'].astype(float)
    

    column_names = {
        'lid': 'Location ID',
        'name': 'Name',
        'lat': 'Latitude',
        'lon': 'Longitude',
        'wfo': 'Weather Forecast Office',
        'rfc': 'River Forecast Center',
        'wrr': 'Water Resource Region',
        'state': 'State',
        'minor': 'Minor Flood Risk',
        'moderate': 'Moderate Flood Risk',
        'major': 'Major Flood Risk',
        'date_produced': 'Date Produced',
        'obj_or_subj': 'Objective or Subjective',
        'interval': 'Interval'
    }
    data.rename(columns=column_names, inplace=True)
    

    return data

api_url = 'https://water.weather.gov/resources/tmp/long_range_river_flood_risk_data.csv' 
csv_data = download_dataset(api_url)
processed_data = process_dataset(csv_data)

processed_data['Longitude'] = -processed_data['Longitude']

## Plotting graph 1


In [4]:
# def plot_data(processed_data):
#     plt.figure(figsize=(10, 6))
#     plt.bar(processed_data['Name'][:10], processed_data['Minor Flood Risk'][:10], color='#9ecae1', label='Minor Flood Risk')
#     plt.bar(processed_data['Name'][:10], processed_data['Moderate Flood Risk'][:10], color='#4292c6', label='Moderate Flood Risk', bottom=processed_data['Minor Flood Risk'][:10])
#     plt.bar(processed_data['Name'][:10], processed_data['Major Flood Risk'][:10], color='#084594', label='Major Flood Risk',
#             bottom=processed_data['Minor Flood Risk'][:10] + processed_data['Moderate Flood Risk'][:10])
#     plt.xlabel('Location')
#     plt.ylabel('Flood Risk Levels')
#     plt.title('Flood Risk Levels by Location')
#     plt.xticks(rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

## Plotting graph 2

In [5]:
# def display_flood_risk_pie_chart(processed_data):
#     risk_levels_sum = {
#         'Minor Flood Risk': processed_data['Minor Flood Risk'].sum(),
#         'Moderate Flood Risk': processed_data['Moderate Flood Risk'].sum(),
#         'Major Flood Risk': processed_data['Major Flood Risk'].sum()
#     }

#     labels = list(risk_levels_sum.keys())
#     sizes = list(risk_levels_sum.values())
#     total = sum(sizes)
#     percentages = [(s/total)*100 for s in sizes]  # Convert to percentages

#     pie = Pie(sizes=percentages, labels=labels, sort=False, display_labels='outside',
#               label_color='black', colors=['#9ecae1', '#6baed6', '#4292c6'], 
#               label_display_format='.2f')

#     tooltip = Tooltip(fields=['size'], labels=['Percentage'])
#     pie.tooltip = tooltip
#     pie.tooltip_formats = ['.2f%']  # Format percentages with 2 decimal places

#     fig = Figure(marks=[pie], title='Distribution of Flood Risk Levels Across All Locations')

#     return VBox([fig])

## Calling all the graphs

In [6]:
# plot_data(processed_data)
# display_flood_risk_pie_chart(processed_data)

## Plotting graph 3

In [7]:
def update_markers(risk_option, state_option):
    mymap = folium.Map(location=[processed_data['Latitude'].mean(), processed_data['Longitude'].mean()], zoom_start=5)
    for index, row in processed_data.iterrows():
        if row[risk_option] >= 0.06 and risk_option == 'Major Flood Risk' and row['State'] == state_option:
            color = 'darkblue'
            folium.Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color=color)).add_to(mymap)
        elif row[risk_option] >= 0.075 and risk_option == 'Moderate Flood Risk' and row['State'] == state_option:
            color = 'blue'
            folium.Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color=color)).add_to(mymap)
        elif risk_option == 'Minor Flood Risk' and row['State'] == state_option:
            color = 'lightblue'
            folium.Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color=color)).add_to(mymap)
    display(mymap)

def main():
    risk_options = ['Minor Flood Risk', 'Moderate Flood Risk', 'Major Flood Risk']
    state_options = processed_data['State'].unique().tolist()
    risk_dropdown = widgets.Dropdown(options=risk_options, description='Flood Risk Level')
    state_dropdown = widgets.Dropdown(options=state_options, description='State')

    interactive_plot = widgets.interactive(update_markers, risk_option=risk_dropdown, state_option=state_dropdown)
    display(interactive_plot) 

## Calling graph 3

In [8]:
# main()

## Interactive graph

In [9]:
def plot_flood_risk(processed_data, state):
    if state == 'All':
        filtered_data = processed_data
    else:
        filtered_data = processed_data[processed_data['State'] == state]
    
    plt.figure(figsize=(20, 15))
    

    risk_levels_sum = {
        'Minor Flood Risk': filtered_data['Minor Flood Risk'].sum(),
        'Moderate Flood Risk': filtered_data['Moderate Flood Risk'].sum(),
        'Major Flood Risk': filtered_data['Major Flood Risk'].sum()
    }

    labels = list(risk_levels_sum.keys())
    sizes = list(risk_levels_sum.values())
    total = sum(sizes)
    percentages = [(s/total)*100 for s in sizes]  # Convert to percentages

    plt.subplot(3, 2, 1)
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=['#9ecae1', '#6baed6', '#4292c6'])
    plt.title('Distribution of Flood Risk Levels Across The Given State')
    
    plt.subplot(3, 2, 2)
    plt.hist(filtered_data['Minor Flood Risk'], bins=10, edgecolor='black', color='#9ecae1')
    plt.xlabel('Minor Flood Risk')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Minor Flood Risk for {state} State')

    plt.subplot(3, 2, 3)
    plt.hist(filtered_data['Moderate Flood Risk'], bins=10, edgecolor='black', color='#6baed6')
    plt.xlabel('Moderate Flood Risk')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Moderate Flood Risk for {state} State')

    plt.subplot(3, 2, 4)
    plt.hist(filtered_data['Major Flood Risk'], bins=10, edgecolor='black', color = '#4292c6')
    plt.xlabel('Major Flood Risk')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Major Flood Risk for {state} State')

    plt.subplot(3, 1, 3)
    locations = filtered_data['Name'][:10]
    minor_risk = filtered_data['Minor Flood Risk'][:10]
    moderate_risk = filtered_data['Moderate Flood Risk'][:10]
    major_risk = filtered_data['Major Flood Risk'][:10]

    plt.bar(locations, minor_risk, color='#9ecae1', label='Minor Flood Risk')
    plt.bar(locations, moderate_risk, color='#6baed6', label='Moderate Flood Risk', bottom=minor_risk)
    plt.bar(locations, major_risk, color='#4292c6', label='Major Flood Risk', bottom=minor_risk + moderate_risk)
    plt.xlabel('Location')
    plt.ylabel('Flood Risk Levels')
    plt.title(f'Flood Risk Levels by Location in {state}')
    plt.xticks(rotation=45)
    plt.legend()

    plt.tight_layout()
    plt.show()

state_dropdown = widgets.Dropdown(options=['All'] + processed_data['State'].unique().tolist(), description='State:')

# DASHBOARD 1

In [10]:
widgets.interactive(plot_flood_risk, processed_data=widgets.fixed(processed_data), state=state_dropdown)

interactive(children=(Dropdown(description='State:', options=('All', 'GA', 'MS', 'KS', 'MT', 'NE', 'ND', 'IN',…

# DASHBOARD 2

In [11]:
main()

interactive(children=(Dropdown(description='Flood Risk Level', options=('Minor Flood Risk', 'Moderate Flood Ri…

# Dataset Overview

## Purpose
The dataset is designed to provide detailed insights into flood risks at multiple locations, aiding experts and policymakers in developing strategies for flood preparedness and response.

## Data Fields
- **Location ID (`lid`)**: Unique identifier for each location. (String)
- **Name (`name`)**: Name of the location. (String)
- **Latitude (`lat`)**: Geographic latitude of the location. (Float)
- **Longitude (`lon`)**: Geographic longitude of the location, with negative values indicating the western hemisphere. (Float)
- **Weather Forecast Office (`wfo`)**: The meteorological office responsible for weather forecasts at the location. (String)
- **River Forecast Center (`rfc`)**: The center responsible for issuing river-based forecasts and warnings. (String)
- **Water Resource Region (`wrr`)**: The designated region for water management where the location is situated. (String)
- **State (`state`)**: The U.S. state where the location is found. (String)
- **Minor Flood Risk (`minor`)**: Numeric value representing the assessed risk of minor flooding. (Float)
- **Moderate Flood Risk (`moderate`)**: Numeric value representing the assessed risk of moderate flooding. (Float)
- **Major Flood Risk (`major`)**: Numeric value representing the assessed risk of major flooding. (Float)
- **Date Produced (`date_produced`)**: The date when the data was generated, can be formatted as a date for temporal analyses. (String)
- **Objective or Subjective (`obj_or_subj`)**: Indicates whether the risk assessment is based on objective data or subjective analysis. (String)
- **Interval (`interval`)**: The time interval for which the data applies. (String)

## Data Types
- **Strings**: Used for identifiers, names, and descriptive fields that involve text.
- **Floats**: Used for geographic coordinates and precise risk measurements.
- **Date/String**: 'Date Produced' could be treated as a date type for temporal analysis depending on the formatting.

# Dashboard 1: Flood Risk Analysis

This interactive dashboard is designed to explore and present flood risk data across the U.S. states. It integrates various types of visualizations to provide comprehensive insights into the distribution of flood risks.

## Features

- **State Selection Dropdown**: Allows the user to select either a specific state or 'All' states for aggregate data visualization. This feature facilitates targeted analysis of flood risk data.

- **Visualizations Included**:
  - **Pie Chart**: Displays the proportionate distribution of Minor, Moderate, and Major flood risks, offering an immediate visual assessment of the severity of risks in the selected area.
  - **Histograms**: There are three histograms showing the frequency distributions of Minor, Moderate, and Major flood risks. These plots help understand how commonly different levels of risks occur and their variability within the selected data scope.
  - **Bar Plot**: Compares the top 10 locations within the chosen area by their flood risk levels, displayed in a stacked format. This visualization highlights areas with heightened risks, which can be crucial for prioritizing flood management and mitigation efforts.

## Interactivity

- The dashboard updates dynamically based on the state selected from the dropdown menu, making it a powerful tool for real-time data exploration. All visualizations reflect the current selection, ensuring that users have the latest information at their fingertips.

## Usage

This tool is invaluable for policymakers, urban planners, and disaster management professionals looking to develop informed strategies to mitigate flood risks. It is also a useful educational resource for the public and researchers interested in environmental management.

# Dashboard 2: Interactive Flood Risk Mapping

This dashboard utilizes interactive maps to visualize specific flood risks across various locations in U.S. states. It allows users to select flood risk levels and states to see real-time geographic distributions of risks.

## Features

- **Interactive Map Visualization**: Utilizes Folium for dynamic mapping, providing a geographical representation of flood risk data.
  
- **Customizable Viewing Options**:
  - **Flood Risk Level Dropdown**: Users can choose to view Minor, Moderate, or Major flood risks. This dropdown allows for specific risk level analysis.
  - **State Dropdown**: Users can select a specific state to focus the map's view, enabling more localized analysis of flood risks.

- **Marker Visualization**:
  - **Markers on Map**: Depending on the selected risk level and state, markers are placed on the map in colors representing the severity of flood risks:
    - Dark Blue for Major Flood Risks (threshold >= 0.06)
    - Blue for Moderate Flood Risks (threshold >= 0.075)
    - Light Blue for Minor Flood Risks

### Interactivity

- The map updates dynamically based on user selections from the dropdown menus. This interactivity allows for tailored visualization and aids in pinpointing high-risk areas more effectively.

### Usage

This tool is particularly useful for environmental analysts, emergency planners, and governmental agencies engaged in flood management. It helps in planning interventions and understanding spatial distributions of flood risks.